# Measuring behaviour from cameras and accelerometer

In [ ]:
# Imports
import sys
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from PIL import Image
import actipy  # for reading in accelerometer data

sys.path.append("scripts")
# Local scripts
import autographer

# 1. Prepare camera and accelerometer data

## Camera processing
Plug in the Autographer wearable camera to the computer. 

In [ ]:
# Check the Autographer pops up, i.e. no "No such file or directory" errors
!ls /Volumes/Autographer/DATA

In [ ]:
# Download the data
autographer.downloadData("/Volumes/Autographer/", "raw_data/camera/")

In [ ]:
# If the above was successful probably want to delete the camera data
# autographer.deleteCameraData("/Volumes/Autographer/") # uncomment to run

In [ ]:
# Get timestamps from paths
#             "20231114_182809"
time_format = "%Y%m%d_%H%M%S"


def get_img_times(paths):
    return [datetime.strptime(path.parts[-1][17:32], time_format) for path in paths]

In [ ]:
# Get the list of image file paths
small_img_paths = list(Path("raw_data/camera/small").glob("*.JPG"))
small_img_times = get_img_times(small_img_paths)

tuples = list(zip(small_img_paths, small_img_times))
tuples.sort(key=lambda x: x[0])

small_img_paths, small_img_times = zip(*tuples)

In [ ]:
# Visualise some
n_imgs_to_show = 10

for img_path, img_time in zip(
    small_img_paths[:n_imgs_to_show], small_img_times[:n_imgs_to_show]
):
    plt.imshow(Image.open(img_path))
    plt.title(img_time.strftime("%Hh%Mm%Ss"))
    plt.show()

## Accelerometer processing
Plug in the Axivity

In [ ]:
# Check the Autographer pops up, i.e. something like AX317_43923 under /Volumes
!ls /Volumes

In [ ]:
%%bash
# Copy the .CWA file to raw_data/accelerometer
mkdir raw_data/accelerometer
mv /Volumes/AX317_43923/CWA-DATA.CWA raw_data/accelerometer/CWA-DATA.CWA
ls raw_data/accelerometer

In [ ]:
# Accelerometer reading in
ax3_data, info = actipy.read_device(
    "raw_data/accelerometer/CWA-DATA.CWA",
    lowpass_hz=20,
    calibrate_gravity=True,
    detect_nonwear=True,
    resample_hz=30,
)

In [ ]:
# img
image_datetimes = np.array(small_img_times, dtype=np.datetime64)
image_paths = np.array(small_img_paths)

# axivity
sensor_datetimes = ax3_data.index.to_numpy()
accelerometer_readings = ax3_data[["x", "y", "z"]].to_numpy()
light_readings = ax3_data["light"].to_numpy()
temperature_readings = ax3_data["temperature"].to_numpy()

In [ ]:
class TimeScale:
    def __init__(self, times):
        self.min_time = np.min(times)
        self.max_time = np.max(times)
        self.scale = self.max_time - self.min_time

    def to_unit(self, times):
        """
        datetime -> [0,1]
        """
        return (times - self.min_time) / self.scale

    def to_scale(self, floats):
        """
        [0,1] -> datetime
        """
        return floats * self.scale + self.min_time

    def to_str(self, floats):
        """
        [0,1] -> str
        """
        times = self.to_scale(floats)
        return np.datetime_as_string(times)


def mask_times(
    start_time: np.datetime64,
    duration: np.timedelta64,
    times: np.ndarray[np.datetime64],
    *arrays: np.ndarray
):
    stop_time = start_time + duration
    mask = np.logical_and(times >= start_time, times < stop_time)
    to_return = []
    for array in arrays:
        to_return.append(array[mask])
    return (times[mask], *to_return)

In [ ]:
# Prepare a window of data
start_time = image_datetimes[50]
duration = np.timedelta64(30, "s")

# sub sample given window
(sub_image_datetimes, sub_image_paths) = mask_times(
    start_time, duration, image_datetimes, image_paths
)

(
    sub_sensor_datetimes,
    sub_accelerometer_readings,
    sub_light_readings,
    sub_temperature_readings,
) = mask_times(
    start_time,
    duration,
    sensor_datetimes,
    accelerometer_readings,
    light_readings,
    temperature_readings,
)

# Transform time steps to [0,1]
ts = TimeScale(np.concatenate([sub_image_datetimes, sub_sensor_datetimes]))
sub_image_unittimes = ts.to_unit(sub_image_datetimes)
sub_sensor_unittimes = ts.to_unit(sub_sensor_datetimes)

In [ ]:
# ====== Plot parameters
img_zoom = 0.15  # how large the images appear on the plot
fig, ax = plt.subplots(4, figsize=(10, 10), gridspec_kw={"height_ratios": [1, 1, 1, 1]})
rotation = 0

# Images
for img_path, x in zip(sub_image_paths, sub_image_unittimes):
    ax[0].add_artist(
        AnnotationBbox(
            OffsetImage(Image.open(img_path), zoom=img_zoom), xy=(x, 0.5), frameon=False
        )
    )
ax[0].set_ylabel("Camera", rotation=0, ha="right")
ax[0].set_yticks([])
ax[0].set_xticks(sub_image_unittimes)
str_times = [time[11:19] for time in ts.to_str(sub_image_unittimes)]
ax[0].set_xticklabels(str_times, rotation=rotation)

# Accelerometer
for i in range(3):
    ax[1].plot(
        sub_sensor_unittimes,
        sub_accelerometer_readings[:, i],
        label=f'{["x","y","z"][i]} axis',
    )

ax[1].legend(loc="upper right")
ax[1].set_ylabel("Accelerometer", rotation=0, ha="right")
ax[1].set_xticklabels([])

# Light
ax[2].plot(sub_sensor_unittimes, sub_light_readings)
ax[2].set_ylabel("Light", rotation=0, ha="right")
ax[2].set_xticklabels([])

# Temperature
ax[3].plot(sub_sensor_unittimes, sub_temperature_readings)
ax[3].set_ylabel("Temperature", rotation=0, ha="right")

ticks = np.linspace(0, 1, 10)
str_ticks = [time[11:19] for time in ts.to_str(ticks)]
ax[3].set_xticks(ticks)
ax[3].set_xticklabels(str_ticks, rotation=rotation)

# Adjusting x-axis limits
for a in ax:
    a.set_xlim(-0.1, 1.1)

plt.show()